In [15]:
import tensorflow as tf
import numpy as np

In [9]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense_1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout_1 = tf.layers.dropout(
      inputs=dense_1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    dense_2 = tf.layers.dense(inputs=dropout_1, units=512, activation=tf.nn.relu)
    dropout_2 = tf.layers.dropout(
      inputs=dense_2, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout_2, units=10)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [3]:
'''
train_data = np.load("mnist_train_images.npy")
train_labels = np.load("mnist_train_labels.npy")
eval_data = np.load("mnist_test_images.npy")
eval_labels = np.load("mnist_test_labels.npy")
'''
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [12]:
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="mnist_convnet_model_2")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde6ab54f50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'mnist_convnet_model_2', '_save_summary_steps': 100}


In [5]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=50)


In [13]:
with tf.device('/cpu:0'):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into mnist_convnet_model_2/model.ckpt.
INFO:tensorflow:loss = 2.3283308, step = 1
INFO:tensorflow:global_step/sec: 10.2048
INFO:tensorflow:loss = 2.297419, step = 101 (9.801 sec)
INFO:tensorflow:global_step/sec: 10.7844
INFO:tensorflow:loss = 2.2940958, step = 201 (9.273 sec)
INFO:tensorflow:global_step/sec: 10.6414
INFO:tensorflow:loss = 2.2999237, step = 301 (9.397 sec)
INFO:tensorflow:global_step/sec: 10.5789
INFO:tensorflow:loss = 2.2728722, step = 401 (9.453 sec)
INFO:tensorflow:global_step/sec: 10.2815
INFO:tensorflow:loss = 2.2562985, step = 501 (9.726 sec)
INFO:tensorflow:global_step/sec: 10.5171
INFO:tensorflow:loss = 2.2444978, step = 601 (9.508 sec)
INFO:tensorflow:global_step/sec: 10.2959
INFO:tensorf

INFO:tensorflow:loss = 0.22055088, step = 7901 (10.223 sec)
INFO:tensorflow:global_step/sec: 9.94308
INFO:tensorflow:loss = 0.31027693, step = 8001 (10.057 sec)
INFO:tensorflow:global_step/sec: 10.0195
INFO:tensorflow:loss = 0.29718944, step = 8101 (9.981 sec)
INFO:tensorflow:global_step/sec: 10.0289
INFO:tensorflow:loss = 0.20913118, step = 8201 (9.971 sec)
INFO:tensorflow:global_step/sec: 10.0419
INFO:tensorflow:loss = 0.29947525, step = 8301 (9.958 sec)
INFO:tensorflow:global_step/sec: 9.97304
INFO:tensorflow:loss = 0.38797206, step = 8401 (10.027 sec)
INFO:tensorflow:global_step/sec: 9.97883
INFO:tensorflow:loss = 0.22061557, step = 8501 (10.021 sec)
INFO:tensorflow:global_step/sec: 9.86689
INFO:tensorflow:loss = 0.27658856, step = 8601 (10.135 sec)
INFO:tensorflow:global_step/sec: 10.0977
INFO:tensorflow:loss = 0.2521375, step = 8701 (9.903 sec)
INFO:tensorflow:global_step/sec: 10.1523
INFO:tensorflow:loss = 0.25789288, step = 8801 (9.850 sec)
INFO:tensorflow:global_step/sec: 9.94

INFO:tensorflow:global_step/sec: 10.4634
INFO:tensorflow:loss = 0.1683589, step = 16001 (9.557 sec)
INFO:tensorflow:global_step/sec: 10.778
INFO:tensorflow:loss = 0.18404636, step = 16101 (9.278 sec)
INFO:tensorflow:global_step/sec: 11.0928
INFO:tensorflow:loss = 0.1220083, step = 16201 (9.015 sec)
INFO:tensorflow:global_step/sec: 11.1076
INFO:tensorflow:loss = 0.12165005, step = 16301 (9.003 sec)
INFO:tensorflow:global_step/sec: 11.1878
INFO:tensorflow:loss = 0.24112405, step = 16401 (8.938 sec)
INFO:tensorflow:global_step/sec: 11.3151
INFO:tensorflow:loss = 0.1764725, step = 16501 (8.838 sec)
INFO:tensorflow:global_step/sec: 11.2156
INFO:tensorflow:loss = 0.122112885, step = 16601 (8.916 sec)
INFO:tensorflow:global_step/sec: 11.3641
INFO:tensorflow:loss = 0.22115776, step = 16701 (8.800 sec)
INFO:tensorflow:global_step/sec: 11.0631
INFO:tensorflow:loss = 0.15141377, step = 16801 (9.039 sec)
INFO:tensorflow:global_step/sec: 10.9696
INFO:tensorflow:loss = 0.21069126, step = 16901 (9.11

In [14]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-23-14:37:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from mnist_convnet_model_2/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-23-14:37:55
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9679, global_step = 20000, loss = 0.10220424
{'loss': 0.10220424, 'global_step': 20000, 'accuracy': 0.9679}
